In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [ ]:
# Cargar los datos (reemplaza 'nombre_del_archivo.csv' con el nombre de tu archivo de datos)
data = pd.read_csv('nombre_del_archivo.csv')

# Paso 2: Dividir los datos en características (X) y etiquetas (y)
X = data.drop('etiqueta', axis=1)
y = data['etiqueta']

# Paso 3: Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 4: Crear un pipeline con modelos para comparar y seleccionar
models = [('Logistic Regression', LogisticRegression()),
          ('Support Vector Machine', SVC(probability=True)),
          ('Random Forest', RandomForestClassifier())]

results = []
names = []

for name, model in models:
    pipeline = Pipeline([('scaler', StandardScaler()), (name, model)])
    
    # Entrenar el modelo por batch (opcional)
    # Tamaño del minibatch (batch_size)
    batch_size = 32
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]
        pipeline.fit(X_batch, y_batch)
    
    # Entrenar el modelo completo en el conjunto de entrenamiento
    pipeline.fit(X_train, y_train)
    
    # Validación cruzada (opcional)
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5)
    
    # Evaluar el modelo en el conjunto de prueba
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
    
    # Métricas de rendimiento
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    results.append(roc_auc)
    names.append(name)
    
    print(f"Modelo: {name}")
    print("ROC AUC:", roc_auc)
    print("CV Scores:", cv_scores)
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print("-" * 50)

# Paso 5: Comparar y seleccionar el mejor modelo
best_model_name = names[np.argmax(results)]
best_model = [model[1] for model in models if model[0] == best_model_name][0]

print(f"Mejor Modelo: {best_model_name}")

# Paso 6: Entrenar el mejor modelo en el conjunto de entrenamiento completo
best_pipeline = Pipeline([('scaler', StandardScaler()), (best_model_name, best_model)])
best_pipeline.fit(X_train, y_train)

# Paso 7: Evaluar el mejor modelo en el conjunto de prueba
y_pred = best_pipeline.predict(X_test)
y_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]

print(f"Mejor Modelo: {best_model_name}")
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Paso 8: Gráficas para observar el rendimiento del mejor modelo
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'{best_model_name} (area = {roc_auc_score(y_test, y_pred_proba):.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()